In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('data/run1.csv')
df.head(2)

,gate_type,voltage,thickness,clock_cycle,t_rise,t_fall,t_delay
0,buffer,0.3,0.1,100,3.131324e-12,3.17279e-12,9.921646e-12
1,buffer,0.4,0.1,100,8.473565e-13,1.456597e-12,7.269524e-12


In [ ]:
def get_gate(gate_type='buffer'): return df[df.gate_type == gate_type]

In [ ]:
get_gate()

,gate_type,voltage,thickness,clock_cycle,t_rise,t_fall,t_delay
0,buffer,0.3,0.1,100,3.131324e-12,3.17279e-12,9.921646e-12
1,buffer,0.4,0.1,100,8.473565e-13,1.456597e-12,7.269524e-12
2,buffer,0.5,0.1,100,5.938628e-13,1.375314e-12,6.446369e-12
3,buffer,0.6,0.1,100,5.413394e-13,1.283254e-12,6.067873e-12
4,buffer,0.7,0.1,100,5.900118e-13,1.28435e-12,5.955801e-12
...,...,...,...,...,...,...,...
1129,buffer,0.5,9.0,900,2.104876e-12,FAILED,2.848981e-11
1130,buffer,0.6,9.0,900,2.209573e-12,FAILED,3.062062e-11
1131,buffer,0.7,9.0,900,2.736121e-12,FAILED,3.267445e-11
1132,buffer,0.8,9.0,900,3.687548e-12,FAILED,3.427408e-11


In [ ]:
unique_gates = list(df.gate_type.unique())
unique_gates

['buffer',
 'inv_x1',
 'inv_x2',
 'inv_x4',
 'nand2',
 'nand3',
 'nand4',
 'nor2',
 'nor3',
 'nor4']

In [ ]:
def get_gate_xy(df, gate_type='buffer'):
    x = np.vstack([df.clock_cycle, df.thickness, df.voltage]).T
    y = np.vstack([df.t_delay, df.t_fall, df.t_rise]).T
    return x, y

In [ ]:
x, y = get_gate_xy(df)
x.shape, y.shape

((11340, 3), (11340, 3))

In [ ]:
df[df.t_rise != ' FAILED'].shape, df[df.t_delay != ' FAILED'].shape, df.shape

((2268, 7), (11340, 7), (11340, 7))

In [ ]:
gate_dict = dict.fromkeys(unique_gates)
gate_dict

{'buffer': None,
 'inv_x1': None,
 'inv_x2': None,
 'inv_x4': None,
 'nand2': None,
 'nand3': None,
 'nand4': None,
 'nor2': None,
 'nor3': None,
 'nor4': None}

In [ ]:
for gate in unique_gates: 
    gate_dict[gate] = get_xy_df(df, gate_type=gate)

In [ ]:
gate_dict['buffer'][0].shape

(11340, 3)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [ ]:
def ridge_reg(x_train, y_train, x_valid, y_valid):
    print('Doing Ridge Regression.....')
    ridge = Ridge(alpha = 1)
    ridge.fit(x_train, y_train)
    print('training score = ', ridge.score(x_train, y_train))
    y_pred_ridge = ridge.predict(x_valid)
    mse_ridge = mean_squared_error(y_valid, y_pred_ridge) 
    print('test score =', ridge.score(x_valid, y_valid)) 

def do_rf(x_train, y_train, x_valid, y_valid):
    print('\n\ndoing RF.....')
    RF = RandomForestRegressor(max_depth=100, random_state=0)
    RF.fit(x_train, y_train)
    print('trainig score = ', RF.score(x_train, y_train))
    y_pred = RF.predict(x_valid)
    mse = mean_squared_error(y_valid, y_pred)
    r2 = r2_score(y_valid, y_pred) # coefficient of determination
    print('test score =', RF.score(x_valid, y_valid))
    print(f'the mse is {mse}')
    print(f"the r2 score is: {r2}")
    return y_pred

def train_models(x, y, random_state=0):
    x_train, x_valid, y_train, y_valid = train_test_split(x, y, 
                                                        random_state=random_state, 
                                                        test_size=0.1)    
    ridge_reg(x_train, y_train, x_valid, y_valid)
    y_pred = do_rf(x_train, y_train, x_valid, y_valid)
    return y_pred

### Lets start with our Buffer circuit

In [ ]:
_ = train_models(X_buffer, y_buffer, random_state=123)

Doing Ridge Regression.....
training score =  0.9167102409624371
test score = 0.8967879854390292


doing RF.....
trainig score =  -3.6984307936194227e-06
test score = -0.0005074970005255105
the mse is 4.938222597074191e-23


### Inverter

In [ ]:
_ = train_models(X_inverter, y_inverter, random_state=0)

Doing Ridge Regression.....
training score =  0.9068440076824577
test score = 0.9137210104799065


doing RF.....
trainig score =  -8.346795311882834e-06
test score = -0.01963368652833042
the mse is 4.9345836306404944e-23


### Nand

In [ ]:
_ = train_models(X_nand, y_nand, random_state=1)

Doing Ridge Regression.....
training score =  0.8901950211342888
test score = 0.8801793859431911


doing RF.....
trainig score =  -8.62405818091716e-06
test score = -0.0011959744121536147
the mse is 4.134560228860396e-23


### Nor

In [ ]:
_ = train_models(X_nor, y_nor)

Doing Ridge Regression.....
training score =  0.8899918684137367
test score = 0.898750119906659


doing RF.....
trainig score =  -9.065795662843712e-06
test score = -0.018704305257992344
the mse is 3.088929916154021e-23


# Normalizing the dependent variable helps?

Looks like things are not working as expected out of the box! Normally you do not need to do much with the data with a random-forest-classifier. but apparently this is not the case with regressors.....

In [ ]:
_ = train_models(X_nor, y_nor/y_nor.min())

Doing Ridge Regression.....
training score =  0.889991868413737
test score = 0.8987501199066591


doing RF.....
trainig score =  0.9994965217624411
test score = 0.997362112592557
the mse is 0.001056661204020164
the r2 score is: 0.997362112592557


So by normalizing the dependent values we can replicate the previous Random Forest Model perforamnce....

In [ ]:
_ = train_models(X_buffer, y_buffer/y_buffer.min())

Doing Ridge Regression.....
training score =  0.9151547232450441
test score = 0.9096239937163108


doing RF.....
trainig score =  0.9996718330442516
test score = 0.9981174754219625
the mse is 0.06108506176995491
the r2 score is: 0.9981174754219625


In [ ]:
_ = train_models(X_inverter, y_inverter / y_inverter.min())

Doing Ridge Regression.....
training score =  0.9068440076824578
test score = 0.9137210104799065


doing RF.....
trainig score =  0.9995370899090714
test score = 0.9973107488297214
the mse is 0.18724402530283232
the r2 score is: 0.9973107488297214


In [ ]:
_ = train_models(X_nand, y_nand / y_nand.min())

Doing Ridge Regression.....
training score =  0.8869122400572241
test score = 0.9092443535388508


doing RF.....
trainig score =  0.9994631916438235
test score = 0.9970597571271802
the mse is 0.0711064052789472
the r2 score is: 0.9970597571271802
